In [1]:
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
from langchain.agents import initialize_agent

load_dotenv()

False

In [2]:
open_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(
    openai_api_key = open_api_key,
    temperature=0,
    model_name="gpt-4-0125-preview"
)

/home/jason/.local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
from langchain.agents import tool

@tool
def diet_planner(restriction):
    '''Uses an API to find the ideal diet plan given your inputs'''
    return f"diet planner used with the given input: {restriction}, eat 50 pounds of greens a day"

@tool
def workout_planner(weight):
    '''Uses an API to find the ideal workout plan given your inputs'''
    return f"workout_planner used with the given input: {weight}, do 50 situps a day"

tools = [diet_planner, workout_planner]

In [4]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage

def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [5]:
def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [6]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

members = ["Fitness trainer", "Dietician"]
system_prompt = (
    " You are a supervisor tasked with delagating tasks to the fitness trainer and the dietician. "
    " Tell the fitness trainer to consider user inputs, and devise a workout plan without using any tools given it. "
    " Once the fitness trainer has given a response, tell the dietician to consider user inputs, and devise a diet plan without using any tools. "
    " Once both the fitness trainer and dietician has given a response, respond with finish."
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the prompt and the conversation, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [7]:
import operator
from typing import Annotated, Any, Dict, List, Optional, Sequence, TypedDict
import functools

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END


# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str


fitness_agent = create_agent(llm, [workout_planner], "You are a professional workout instructor")
fitness_node = functools.partial(agent_node, agent=fitness_agent, name="Fitness trainer")
diet_agent = create_agent(llm, [diet_planner], "You are a professional dietician")
diet_node = functools.partial(agent_node, agent= diet_agent, name="Dietician")


workflow = StateGraph(AgentState)
workflow.add_node("Dietician", diet_node)
workflow.add_node("Fitness trainer", fitness_node)
workflow.add_node("supervisor", supervisor_chain)

In [8]:
for member in members:
    # We want our workers to ALWAYS "report back" to the supervisor when done
    workflow.add_edge(member, "supervisor")
# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
# Finally, add entrypoint
workflow.set_entry_point("supervisor")

graph = workflow.compile()


In [9]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="99 pounds, vegan")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'supervisor': {'next': 'Fitness trainer'}}
----
{'Fitness trainer': {'messages': [HumanMessage(content="For someone weighing 99 pounds, an ideal workout plan would include doing 50 situps a day. Since you mentioned being vegan, it's also important to ensure that your diet supports your workout goals. Make sure to include a variety of plant-based protein sources, such as lentils, chickpeas, tofu, and quinoa, to aid in muscle recovery and growth. Stay hydrated and consider integrating other forms of exercise, like yoga or light jogging, to maintain a balanced fitness routine.", name='Fitness trainer')]}}
----
{'supervisor': {'next': 'Dietician'}}
----
{'Dietician': {'messages': [HumanMessage(content="That sounds like a solid plan! Incorporating a variety of exercises along with a balanced vegan diet can definitely help in achieving your fitness goals. Here are a few additional tips to consider:\n\n1. **Protein Intake**: As a vegan, it's crucial to pay attention to your protein intake to

In [10]:
graph.invoke({"messages": ["99 pounds,vegan"]})

{'messages': ['99 pounds,vegan',
  HumanMessage(content="For someone weighing 99 pounds, a recommended workout plan is to do 50 situps a day. Since you mentioned being vegan, it's also important to ensure your diet supports your fitness goals. Focus on a balanced intake of plant-based proteins, carbohydrates, and fats to fuel your workouts and recovery.", name='Fitness trainer'),
  HumanMessage(content="For someone weighing 99 pounds and following a vegan lifestyle, it's crucial to maintain a balanced diet that supports your fitness goals, especially if you're incorporating a workout plan that includes 50 situps a day. Here's a general guideline to help you align your diet with your fitness routine:\n\n### Protein\nProtein is essential for muscle repair and growth. As a vegan, you can get your protein from sources like:\n- Lentils and beans\n- Tofu and tempeh\n- Quinoa and other whole grains\n- Nuts and seeds (e.g., almonds, chia seeds, hemp seeds)\n- Pea protein powders\n\nAim for abo